In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Sample dataset (you'll want to expand this with real digit encodings)
# Each entry: (5x4 binary image flattened, label
digits = {
    0: [
        [0,1,1,0],
        [1,0,0,1],
        [1,0,0,1],
        [1,0,0,1],
        [0,1,1,0]
    ],
    1: [
        [0,0,1,0],
        [0,1,1,0],
        [0,0,1,0],
        [0,0,1,0],
        [0,1,1,1]
    ],
    2: [
        [0,1,1,0],
        [1,0,0,1],
        [0,0,1,0],
        [0,1,0,0],
        [1,1,1,1]
    ],
    3: [
        [1,1,1,0],
        [0,0,0,1],
        [0,0,1,0],
        [0,0,0,1],
        [1,1,1,0]
    ],
    4: [
        [0,0,1,0],
        [0,1,1,0],
        [1,0,1,0],
        [1,1,1,1],
        [0,0,1,0]
    ],
    5: [
        [1,1,1,1],
        [1,0,0,0],
        [1,1,1,0],
        [0,0,0,1],
        [1,1,1,0]
    ],
    6: [
        [0,1,1,1],
        [1,0,0,0],
        [1,1,1,0],
        [1,0,0,1],
        [0,1,1,0]
    ],
    7: [
        [1,1,1,1],
        [0,0,0,1],
        [0,0,1,0],
        [0,1,0,0],
        [0,1,0,0]
    ],
    8: [
        [0,1,1,0],
        [1,0,0,1],
        [0,1,1,0],
        [1,0,0,1],
        [0,1,1,0]
    ],
    9: [
        [0,1,1,0],
        [1,0,0,1],
        [0,1,1,1],
        [0,0,0,1],
        [1,1,1,0]
    ]
}


In [3]:
X = []
y = []

for digit, matrix in digits.items():
    flat = [pixel for row in matrix for pixel in row]
    X.append(flat)
    y.append(digit)

X_tensor = torch.tensor(X, dtype=torch.float)
y_tensor = torch.tensor(y, dtype=torch.long)

print("Input shape:", X_tensor.shape)  # (10 digitos, 20 bits)
print("Labels:", y_tensor)

Input shape: torch.Size([10, 20])
Labels: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [4]:
# One-hot encode labels
y_onehot = F.one_hot(y_tensor, num_classes=10).float()


# transformar a variável-alvo em uma matriz 10 x 10

In [13]:
y_onehot

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [5]:
# Definir uma simples Rede Neural
class DigitNet(nn.Module):
    def __init__(self, hidden_layer):
        super(DigitNet, self).__init__()
        self.layer1 = nn.Linear(20, hidden_layer)  # 20 input features (bits por digito) → 32 camada oculta
        self.layer2 = nn.Linear(hidden_layer, 10)  # 32 oculta → 10 output classes

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = self.layer2(x)
        return F.softmax(x, dim=1) # retornar probabilidades usando a função de ativação softmax


In [ ]:
# Numero de nêurons na camada oculta
hidden_layer = 35

model = DigitNet(hidden_layer=hidden_layer)
criterion = nn.MSELoss() # Função de Custo
optimizer = optim.SGD(params=model.parameters(), lr=0.1, momentum=0.4)

# Training loop
for epoch in range(10001):
    optimizer.zero_grad()
    outputs = model(X_tensor)
    loss = criterion(outputs, y_onehot)
    loss.backward()
    optimizer.step()
    if (epoch) % 10 == 0:
        print(f"Epoch {epoch} - Loss: {loss.item():.4f}")

# Example prediction
with torch.no_grad():
    test_digit = torch.tensor([[0,0,1,0, 
                                1,0,1,1, # digito 0 com 3 bits errados para verificar se consegue fazer predição corretamente
                                1,0,0,1, 
                                1,0,0,1, 
                                1,1,1,0]], dtype=torch.float)
    prediction = model(test_digit)
    predicted_class = prediction.argmax(dim=1).item()
    print(f"Predicted digit: {predicted_class}")
    
    
# Com 35 neurons na camada oculta, lr de 0.1, momentum de 0.4, atingimos um resultado ideal de MSE = 0.0001 para 10.000 épocas


Epoch 0 - Loss: 0.0906
Epoch 10 - Loss: 0.0905
Epoch 20 - Loss: 0.0904
Epoch 30 - Loss: 0.0903
Epoch 40 - Loss: 0.0901
Epoch 50 - Loss: 0.0900
Epoch 60 - Loss: 0.0899
Epoch 70 - Loss: 0.0898
Epoch 80 - Loss: 0.0897
Epoch 90 - Loss: 0.0895
Epoch 100 - Loss: 0.0894
Epoch 110 - Loss: 0.0893
Epoch 120 - Loss: 0.0892
Epoch 130 - Loss: 0.0890
Epoch 140 - Loss: 0.0889
Epoch 150 - Loss: 0.0888
Epoch 160 - Loss: 0.0887
Epoch 170 - Loss: 0.0885
Epoch 180 - Loss: 0.0884
Epoch 190 - Loss: 0.0883
Epoch 200 - Loss: 0.0882
Epoch 210 - Loss: 0.0880
Epoch 220 - Loss: 0.0879
Epoch 230 - Loss: 0.0878
Epoch 240 - Loss: 0.0876
Epoch 250 - Loss: 0.0875
Epoch 260 - Loss: 0.0873
Epoch 270 - Loss: 0.0872
Epoch 280 - Loss: 0.0870
Epoch 290 - Loss: 0.0869
Epoch 300 - Loss: 0.0868
Epoch 310 - Loss: 0.0866
Epoch 320 - Loss: 0.0865
Epoch 330 - Loss: 0.0863
Epoch 340 - Loss: 0.0862
Epoch 350 - Loss: 0.0860
Epoch 360 - Loss: 0.0859
Epoch 370 - Loss: 0.0857
Epoch 380 - Loss: 0.0856
Epoch 390 - Loss: 0.0854
Epoch 400 -

In [ ]:
# Numero de nêurons na camada oculta
hidden_layer = 15

model = DigitNet(hidden_layer=hidden_layer)
criterion = nn.MSELoss() # Função de Custo
optimizer = optim.SGD(params=model.parameters(), lr=0.9, momentum=0.0)

# Training loop
for epoch in range(10001):
    optimizer.zero_grad()
    outputs = model(X_tensor)
    loss = criterion(outputs, y_onehot)
    loss.backward()
    optimizer.step()
    if (epoch) % 10 == 0:
        print(f"Epoch {epoch} - Loss: {loss.item():.4f}")

# Example prediction
with torch.no_grad():
    test_digit = torch.tensor([[0,0,1,0, 
                                1,0,1,1, # digito 0 com 3 bits errados para verificar se consegue fazer predição corretamente
                                1,0,0,1, 
                                1,0,0,1, 
                                1,1,1,0]], dtype=torch.float)
    prediction = model(test_digit)
    predicted_class = prediction.argmax(dim=1).item()
    print(f"Predicted digit: {predicted_class}")
    
    
# Com 15 neurons na camada oculta, lr de 0.9, momentum de 0.0, atingimos um resultado ideal de MSE = 0.0000 para 10.000 épocas
# Mesmo com learning rate alta, o modelo conseguiu convergir.


Epoch 0 - Loss: 0.0910
Epoch 10 - Loss: 0.0905
Epoch 20 - Loss: 0.0900
Epoch 30 - Loss: 0.0895
Epoch 40 - Loss: 0.0890
Epoch 50 - Loss: 0.0885
Epoch 60 - Loss: 0.0880
Epoch 70 - Loss: 0.0874
Epoch 80 - Loss: 0.0868
Epoch 90 - Loss: 0.0862
Epoch 100 - Loss: 0.0855
Epoch 110 - Loss: 0.0846
Epoch 120 - Loss: 0.0837
Epoch 130 - Loss: 0.0827
Epoch 140 - Loss: 0.0816
Epoch 150 - Loss: 0.0805
Epoch 160 - Loss: 0.0793
Epoch 170 - Loss: 0.0781
Epoch 180 - Loss: 0.0767
Epoch 190 - Loss: 0.0752
Epoch 200 - Loss: 0.0736
Epoch 210 - Loss: 0.0719
Epoch 220 - Loss: 0.0699
Epoch 230 - Loss: 0.0677
Epoch 240 - Loss: 0.0653
Epoch 250 - Loss: 0.0626
Epoch 260 - Loss: 0.0598
Epoch 270 - Loss: 0.0569
Epoch 280 - Loss: 0.0538
Epoch 290 - Loss: 0.0506
Epoch 300 - Loss: 0.0471
Epoch 310 - Loss: 0.0436
Epoch 320 - Loss: 0.0400
Epoch 330 - Loss: 0.0366
Epoch 340 - Loss: 0.0333
Epoch 350 - Loss: 0.0303
Epoch 360 - Loss: 0.0276
Epoch 370 - Loss: 0.0251
Epoch 380 - Loss: 0.0229
Epoch 390 - Loss: 0.0210
Epoch 400 -

In [7]:
# Numero de nêurons na camada oculta
hidden_layer = 25

model = DigitNet(hidden_layer=hidden_layer)
criterion = nn.MSELoss() # Função de Custo
optimizer = optim.SGD(params=model.parameters(), lr=0.9, momentum=0.4)

# Training loop
for epoch in range(10001):
    optimizer.zero_grad()
    outputs = model(X_tensor)
    loss = criterion(outputs, y_onehot)
    loss.backward()
    optimizer.step()
    if (epoch) % 10 == 0:
        print(f"Epoch {epoch} - Loss: {loss.item():.4f}")

# Example prediction
with torch.no_grad():
    test_digit = torch.tensor([[0,0,1,0, 
                                1,0,1,1, # digito 0 com 3 bits errados para verificar se consegue fazer predição corretamente
                                1,0,0,1, 
                                1,0,0,1, 
                                1,1,1,0]], dtype=torch.float)
    prediction = model(test_digit)
    predicted_class = prediction.argmax(dim=1).item()
    print(f"Predicted digit: {predicted_class}")
    
    
# Com 25 neurons na camada oculta, lr de 0.9, momentum de 0.4, atingimos um resultado ideal de MSE = 0.0000 para 10.000 épocas


Epoch 0 - Loss: 0.0903
Epoch 10 - Loss: 0.0893
Epoch 20 - Loss: 0.0883
Epoch 30 - Loss: 0.0872
Epoch 40 - Loss: 0.0860
Epoch 50 - Loss: 0.0846
Epoch 60 - Loss: 0.0831
Epoch 70 - Loss: 0.0811
Epoch 80 - Loss: 0.0787
Epoch 90 - Loss: 0.0757
Epoch 100 - Loss: 0.0720
Epoch 110 - Loss: 0.0674
Epoch 120 - Loss: 0.0617
Epoch 130 - Loss: 0.0556
Epoch 140 - Loss: 0.0497
Epoch 150 - Loss: 0.0442
Epoch 160 - Loss: 0.0391
Epoch 170 - Loss: 0.0340
Epoch 180 - Loss: 0.0291
Epoch 190 - Loss: 0.0246
Epoch 200 - Loss: 0.0206
Epoch 210 - Loss: 0.0169
Epoch 220 - Loss: 0.0136
Epoch 230 - Loss: 0.0110
Epoch 240 - Loss: 0.0089
Epoch 250 - Loss: 0.0072
Epoch 260 - Loss: 0.0059
Epoch 270 - Loss: 0.0050
Epoch 280 - Loss: 0.0042
Epoch 290 - Loss: 0.0036
Epoch 300 - Loss: 0.0031
Epoch 310 - Loss: 0.0027
Epoch 320 - Loss: 0.0024
Epoch 330 - Loss: 0.0022
Epoch 340 - Loss: 0.0019
Epoch 350 - Loss: 0.0018
Epoch 360 - Loss: 0.0016
Epoch 370 - Loss: 0.0015
Epoch 380 - Loss: 0.0014
Epoch 390 - Loss: 0.0013
Epoch 400 -

In [ ]:
# Numero de nêurons na camada oculta
hidden_layer = 25

model = DigitNet(hidden_layer=hidden_layer)
criterion = nn.MSELoss() # Função de Custo
optimizer = optim.SGD(params=model.parameters(), lr=0.1, momentum=0.0)

# Training loop
for epoch in range(10001):
    optimizer.zero_grad()
    outputs = model(X_tensor)
    loss = criterion(outputs, y_onehot)
    loss.backward()
    optimizer.step()
    if (epoch) % 10 == 0:
        print(f"Epoch {epoch} - Loss: {loss.item():.4f}")

# Example prediction
with torch.no_grad():
    test_digit = torch.tensor([[0,0,1,0, 
                                1,0,1,1, # digito 0 com 3 bits errados para verificar se consegue fazer predição corretamente
                                1,0,0,1, 
                                1,0,0,1, 
                                1,1,1,0]], dtype=torch.float)
    prediction = model(test_digit)
    predicted_class = prediction.argmax(dim=1).item()
    print(f"Predicted digit: {predicted_class}")
    
    
# Com 25 neurons na camada oculta, lr de 0.1, momentum de 0.0, atingimos um resultado ideal de MSE = 0.0004 para 10.000 épocas
# Neste caso em particular, com learning rate mais baixa e momentum = 0.0, nota-se que houve uma redução de Loss mais lenta.
# E ainda assim, não alcançou o Loss que os outros modelos acima atingiram, definindo este o modelo de 'pior' performance.
# Com essa taxa de aprendizado mais baixa, o modelo custou mais a aprender os padrões do conjunto de treinamento dado, tendo
# necessidade de muitas épocas para chegar nesse resultado.


Epoch 0 - Loss: 0.0910
Epoch 10 - Loss: 0.0910
Epoch 20 - Loss: 0.0909
Epoch 30 - Loss: 0.0909
Epoch 40 - Loss: 0.0908
Epoch 50 - Loss: 0.0908
Epoch 60 - Loss: 0.0907
Epoch 70 - Loss: 0.0907
Epoch 80 - Loss: 0.0906
Epoch 90 - Loss: 0.0906
Epoch 100 - Loss: 0.0905
Epoch 110 - Loss: 0.0905
Epoch 120 - Loss: 0.0904
Epoch 130 - Loss: 0.0904
Epoch 140 - Loss: 0.0904
Epoch 150 - Loss: 0.0903
Epoch 160 - Loss: 0.0903
Epoch 170 - Loss: 0.0902
Epoch 180 - Loss: 0.0902
Epoch 190 - Loss: 0.0901
Epoch 200 - Loss: 0.0901
Epoch 210 - Loss: 0.0900
Epoch 220 - Loss: 0.0900
Epoch 230 - Loss: 0.0900
Epoch 240 - Loss: 0.0899
Epoch 250 - Loss: 0.0899
Epoch 260 - Loss: 0.0898
Epoch 270 - Loss: 0.0898
Epoch 280 - Loss: 0.0897
Epoch 290 - Loss: 0.0897
Epoch 300 - Loss: 0.0897
Epoch 310 - Loss: 0.0896
Epoch 320 - Loss: 0.0896
Epoch 330 - Loss: 0.0895
Epoch 340 - Loss: 0.0895
Epoch 350 - Loss: 0.0894
Epoch 360 - Loss: 0.0894
Epoch 370 - Loss: 0.0894
Epoch 380 - Loss: 0.0893
Epoch 390 - Loss: 0.0893
Epoch 400 -